In [1]:
import warnings  

import os
import time
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import albumentations as A
from albumentations.pytorch import ToTensorV2
import cv2

import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torch.utils.data.dataset import Subset
from torch.nn.functional import softmax

from efficientnet_pytorch import EfficientNet

from typing import Callable, List, Tuple, Dict
from pathlib import Path

from transformers import AdamW
from transformers import get_cosine_schedule_with_warmup

from sklearn.metrics import roc_auc_score, accuracy_score
from sklearn.model_selection import train_test_split
from collections import defaultdict, OrderedDict
from tqdm.notebook import tqdm
from torchsummary import summary

from fastai2.layers import LabelSmoothingCrossEntropy
from transformers import get_cosine_schedule_with_warmup
from transformers import AdamW

import matplotlib
matplotlib.rcParams.update({'figure.figsize': (16, 12), 'font.size': 14})
import matplotlib.pyplot as plt
%matplotlib inline
from IPython.display import clear_output

In [2]:
warnings.simplefilter("ignore", UserWarning)
warnings.simplefilter("ignore", DeprecationWarning)
warnings.filterwarnings('ignore')
os.environ["PYTHONWARNINGS"] = "ignore"

In [3]:
EXPERIMENT_NAME = "02_efficientnet-b4"

class ConfigExperiment:
    logdir = f"./logs/{EXPERIMENT_NAME}"
    save_dirname = EXPERIMENT_NAME
    submission_file = f"{EXPERIMENT_NAME}.csv"
    seed = 42
    batch_size = 8
    model_name = 'efficientnet-b4'
    size = 512
    num_workers = 20
    root_images = "../../../data/raw/plant-pathology-2020-fgvc7/images/"
    root = "../../../data/raw/plant-pathology-2020-fgvc7/"
    num_classes = 4
    patience = 10
    early_stopping_delta = 1e-4
    num_epochs = 200
    lr = 0.003
    class_names = ["healthy", "multiple_diseases", "rust", "scab"]
    is_fp16_used = False
    
    
def set_seed(seed):
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    np.random.seed(seed)
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    
    
config = ConfigExperiment()
set_seed(config.seed)
config.size = EfficientNet.get_image_size(config.model_name)
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
device = "cuda" if torch.cuda.is_available() else "cpu"

try:
    # Create target Directory
    os.mkdir(config.save_dirname)
    print("Directory " , config.save_dirname ,  " Created ") 
except FileExistsError:
    print("Directory " , config.save_dirname ,  " already exists")

Directory  02_efficientnet-b4  already exists


In [4]:
class PlantDataset(Dataset):
    
    def __init__(self, df, config, transforms=None):
    
        self.df = df
        self.images_dir = config.root_images
        self.class_names = config.class_names
        self.transforms=transforms
        
    def __len__(self):
        return self.df.shape[0]
    
    def __getitem__(self, idx):
        image_src = self.images_dir + self.df.iloc[idx]['image_id'] + '.jpg'
        image = cv2.imread(image_src, cv2.IMREAD_COLOR)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        labels = self.df.iloc[idx][self.class_names].values.astype(np.int8)
        label = torch.argmax(torch.from_numpy(labels))
        
        if self.transforms:
            transformed = self.transforms(image=image)
            image = transformed['image']

        return image, label

In [5]:
def pre_transforms(image_size=224):
    # Convert the image to a square of size image_size x image_size
    # (keeping aspect ratio)
    result = [
        A.LongestMaxSize(max_size=image_size),
        A.PadIfNeeded(image_size, image_size, border_mode=0)
    ]
    
    return result

def hard_transforms():
    result = [
        # Random shifts, stretches and turns with a 50% probability
#         A.RandomResizedCrop(height=config.size, width=config.size, p=1.0),
        A.OneOf([
            A.Rotate(limit=90, p=1),
            A.HorizontalFlip(p=1),
            A.VerticalFlip(p=1),
            A.OpticalDistortion(p=1),
            A.ShiftScaleRotate(rotate_limit=90, shift_limit=0.2, scale_limit=0.1, p=1)
        ], p=0.8),
        
        # Pixels
        A.OneOf([
            A.IAAEmboss(p=1.0),
            A.IAASharpen(p=1.0),
            A.Blur(p=1.0),
        ], p=0.5),

        # Affine
        A.OneOf([
            A.RandomContrast(limit=0.2, p=1),
            A.ElasticTransform(p=1),
            A.IAAPiecewiseAffine(p=1)
        ], p=0.5),
    ]
    
    return result

def post_transforms():
    # we use ImageNet image normalization
    # and convert it to torch.Tensor
    return [A.Normalize(p=1.0), ToTensorV2(p=1.0),]

def compose(transforms_to_compose):
    # combine all augmentations into one single pipeline
    result = A.Compose([item for sublist in transforms_to_compose for item in sublist])
    return result

In [6]:
train_df = pd.read_csv(config.root + 'train.csv')
train, valid = train_test_split(train_df, test_size=0.2, random_state=config.seed, shuffle=True, stratify=train_df[config.class_names])

train_transforms = compose([
    pre_transforms(config.size),
    hard_transforms(), 
    post_transforms()
])
valid_transforms = compose([
    pre_transforms(config.size), 
    post_transforms()
])

show_transforms = compose([
    pre_transforms(config.size),
    hard_transforms()
])

train_dataset = PlantDataset(train, config, train_transforms)
valid_dataset = PlantDataset(valid, config, valid_transforms)

train_dataloader = DataLoader(train_dataset, batch_size=config.batch_size, shuffle=True, num_workers=config.num_workers)
valid_dataloader = DataLoader(valid_dataset, batch_size=config.batch_size, shuffle=False, num_workers=config.num_workers)

In [7]:
def get_model(model_name: str, num_classes: int, pretrained: str = "imagenet") -> EfficientNet:
    model = EfficientNet.from_pretrained(model_name)
    for param in model.parameters():
        param.requires_grad = False
    num_ftrs = model._fc.in_features
    model._fc = nn.Sequential(nn.Linear(num_ftrs, num_classes, bias = True))
    return model

model = get_model(config.model_name, config.num_classes)

Loaded pretrained weights for efficientnet-b4


In [8]:
criterion = LabelSmoothingCrossEntropy()
optimizer = AdamW(model.parameters(), lr=config.lr, weight_decay=1e-3)

num_train_steps = int(len(train_dataset) / config.batch_size * config.num_epochs)
scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=len(train_dataset)/config.batch_size*5, num_training_steps=num_train_steps)

In [9]:
class Trainer:
    def __init__(self, model, train_dataloader: DataLoader, valid_dataloader: DataLoader, criterion, optimizer, scheduler, device, config: ConfigExperiment):
        self.model = model
        self.train_dataloader = train_dataloader
        self.valid_dataloader = valid_dataloader
        self.criterion = criterion
        self.optimizer = optimizer
        self.scheduler = scheduler
        self.device = device
        self.config = config
        self.train_metrics = {
            'avg_loss': [],
            'auc/_mean': [],
            'auc/healthy': [],
            'auc/multiple_diseases': [],
            'auc/rust': [],
            'auc/scab': [],
        }
        self.valid_metrics = {
            'avg_loss': [],
            'auc/_mean': [],
            'auc/healthy': [],
            'auc/multiple_diseases': [],
            'auc/rust': [],
            'auc/scab': [],
        }
        self.counter = 0
        self.delta = config.early_stopping_delta
      
    def run(self):
        self.model.to(device)
        best_valid_loss = float('inf')
        best_valid_auc_mean = 0

        try:
            for i_epoch in tqdm(range(self.config.num_epochs), desc='Epochs', total=config.num_epochs, position=1, leave=True):
                start_time = time.time()

                train_loss, train_outputs, train_targets = self._train()
                valid_loss, valid_outputs, valid_targets = self._evaluate()
                    
                self.train_metrics["avg_loss"].append(train_loss)
                self.train_metrics["auc/_mean"].append(self.comp_metric(train_outputs, train_targets))
                self.train_metrics["auc/healthy"].append(self.healthy_roc_auc(train_outputs, train_targets))
                self.train_metrics["auc/multiple_diseases"].append(self.multiple_diseases_roc_auc(train_outputs, train_targets))
                self.train_metrics["auc/rust"].append(self.rust_roc_auc(train_outputs, train_targets))
                self.train_metrics["auc/scab"].append(self.scab_roc_auc(train_outputs, train_targets))
                
                self.valid_metrics["avg_loss"].append(valid_loss)
                self.valid_metrics["auc/_mean"].append(self.comp_metric(valid_outputs, valid_targets))
                self.valid_metrics["auc/healthy"].append(self.healthy_roc_auc(valid_outputs, valid_targets))
                self.valid_metrics["auc/multiple_diseases"].append(self.multiple_diseases_roc_auc(valid_outputs, valid_targets))
                self.valid_metrics["auc/rust"].append(self.rust_roc_auc(valid_outputs, valid_targets))
                self.valid_metrics["auc/scab"].append(self.scab_roc_auc(valid_outputs, valid_targets))
                
                end_time = time.time()
                epoch_mins, epoch_secs = self._epoch_time(start_time, end_time)
                self.print_progress(i_epoch, epoch_mins, epoch_secs)
                
                self.scheduler.step(self.valid_metrics["auc/_mean"][-1])
                
                if valid_loss < best_valid_loss:
                    self.counter = 0
                    best_valid_loss = valid_loss
                    torch.save(model.state_dict(), f"{config.save_dirname}/best_model_epoch={i_epoch+1}_loss={best_valid_loss}.pth")
                else:
                    self.counter += 1
                    
                if self.valid_metrics["auc/_mean"][-1] > best_valid_auc_mean:
                    best_valid_auc_mean = self.valid_metrics["auc/_mean"][-1]
                    torch.save(model.state_dict(), f"{config.save_dirname}/best_model_epoch={i_epoch+1}_auc_mean={best_valid_auc_mean}.pth")
                    
                if self.counter > self.config.patience:
                    print("EarlyStopping")
                    break
        except KeyboardInterrupt: 
            pass
        
        return self.train_metrics, self.valid_metrics
        
    def _train(self):
        model.train()
        epoch_loss = 0
        epoch_output = None
        epoch_target = None
        for i, (images, labels) in tqdm(enumerate(self.train_dataloader), desc='Train', total=len(self.train_dataloader), position=2, leave=True):
            loss_iten, outputs = self._train_process(images, labels)
            epoch_loss += loss_iten              

            if epoch_output is None:
                epoch_output = outputs.cpu().data
            else:
                epoch_output = torch.cat((epoch_output, outputs.cpu().data))

            if epoch_target is None:
                epoch_target = labels.cpu().data
            else:
                epoch_target = torch.cat((epoch_target, labels.cpu().data))
            
        return epoch_loss / len(self.train_dataloader), epoch_output, epoch_target
    
    def _train_process(self, images, labels):
        images = images.to(self.device)
        labels = labels.to(self.device)
        self.optimizer.zero_grad()
        outputs = self.model(images)
        loss = self.criterion(outputs, labels)
        loss.backward()
        self.optimizer.step()
        return loss.item(), outputs
            
    def _evaluate(self):
        model.eval()
        epoch_loss = 0
        epoch_output = None
        epoch_target = None
        with torch.no_grad():
            for i, (images, labels) in enumerate(self.valid_dataloader):
                images = images.to(device)
                labels = labels.to(device)
                outputs = model(images)
                loss = criterion(outputs, labels)
                epoch_loss += loss.item()
                
                if epoch_output is None:
                    epoch_output = outputs.cpu().data
                else:
                    epoch_output = torch.cat((epoch_output, outputs.cpu().data))

                if epoch_target is None:
                    epoch_target = labels.cpu().data
                else:
                    epoch_target = torch.cat((epoch_target, labels.cpu().data))

        return epoch_loss / len(self.valid_dataloader), epoch_output, epoch_target
    
#     def _evaluate_tta(self):
#         n_samples = len(dataset)
#         y_probas_tta = np.zeros((n_samples, config.num_classes, tta_count), dtype=np.float32)

#         model.eval()
#         model.to(device)
#         with torch.no_grad():
#             for i_epoch in tqdm(range(tta_count), desc='TTA', total=tta_count, position=1, leave=True):
#                 for i, (images, idx) in enumerate(dataloader):
#                     images = images.to(device)
#                     outputs = model(images)
#                     y_pred = F.softmax(outputs, dim=1)

#                     tta_index = i_epoch - 1
#                     start_index = (i % len(dataloader)) * config.batch_size
#                     end_index = min(start_index + config.batch_size, n_samples)
#                     batch_y_probas = y_pred.detach().cpu().numpy()                
#                     y_probas_tta[start_index:end_index, :, tta_index] = batch_y_probas

#         return y_probas_tta

#     def show_history(self):
#         fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(12, 8))
#         clear_output(True)
#         ax[0].plot(history, label='train loss')
#         ax[0].set_xlabel('Batch')
#         ax[0].set_title('Train loss')
#         if train_history is not None:
#             ax[1].plot(train_history, label='general train history')
#             ax[1].set_xlabel('Epoch')
#         if valid_history is not None:
#             ax[1].plot(valid_history, label='general valid history')
#         plt.legend()
#         plt.show()

    
    def _epoch_time(self, start_time, end_time):
        elapsed_time = end_time - start_time
        elapsed_mins = int(elapsed_time / 60)
        elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
        return elapsed_mins, elapsed_secs

    def print_progress(self, i_epoch, epoch_mins, epoch_secs):
        i_epoch = i_epoch + 1
        print(f"Epoch: {i_epoch:02} | Time: {epoch_mins}m {epoch_secs}s")
        print("Training Results - Average Loss: {:.4f} | auc/_mean: {:.4f} | auc/healthy: {:.4f} | auc/multiple_diseases: {:.4f}, auc/rust: {:.4f}, auc/scab: {:.4f}"
            .format(
                self.train_metrics['avg_loss'][-1], 
                self.train_metrics['auc/_mean'][-1],
                self.train_metrics['auc/healthy'][-1],
                self.train_metrics['auc/multiple_diseases'][-1],
                self.train_metrics['auc/rust'][-1],
                self.train_metrics['auc/scab'][-1],
            ))
        print("Evaluating Results - Average Loss: {:.4f} | auc/_mean: {:.4f} | auc/healthy: {:.4f} | auc/multiple_diseases: {:.4f}, auc/rust: {:.4f}, auc/scab: {:.4f}"
            .format( 
                self.valid_metrics['avg_loss'][-1], 
                self.valid_metrics['auc/_mean'][-1],
                self.valid_metrics['auc/healthy'][-1],
                self.valid_metrics['auc/multiple_diseases'][-1],
                self.valid_metrics['auc/rust'][-1],
                self.valid_metrics['auc/scab'][-1],
            ))
        print()
        
    def comp_metric(self, preds, targs, labels=range(4)):
        preds = torch.sigmoid(preds)
        targs = torch.eye(4)[targs]
        return np.mean([roc_auc_score(targs[:,i], preds[:,i]) for i in labels])

    def healthy_roc_auc(self, *args):
        return self.comp_metric(*args, labels=[0])

    def multiple_diseases_roc_auc(self, *args):
        return self.comp_metric(*args, labels=[1])

    def rust_roc_auc(self, *args):
        return self.comp_metric(*args, labels=[2])

    def scab_roc_auc(self, *args):
        return self.comp_metric(*args, labels=[3])



In [10]:
trainer = Trainer(model, train_dataloader, valid_dataloader, criterion, optimizer, scheduler, device, config)
trainer.run();


Epoch: 01 | Time: 0m 35s
Training Results - Average Loss: 1.3792 | auc/_mean: 0.5021 | auc/healthy: 0.4901 | auc/multiple_diseases: 0.5031, auc/rust: 0.4832, auc/scab: 0.5320
Evaluating Results - Average Loss: 1.3789 | auc/_mean: 0.4951 | auc/healthy: 0.5613 | auc/multiple_diseases: 0.4832, auc/rust: 0.3222, auc/scab: 0.6136




Epoch: 02 | Time: 0m 36s
Training Results - Average Loss: 1.3760 | auc/_mean: 0.5157 | auc/healthy: 0.4935 | auc/multiple_diseases: 0.5647, auc/rust: 0.4620, auc/scab: 0.5427
Evaluating Results - Average Loss: 1.3586 | auc/_mean: 0.5548 | auc/healthy: 0.5622 | auc/multiple_diseases: 0.5770, auc/rust: 0.3982, auc/scab: 0.6817




Epoch: 03 | Time: 0m 35s
Training Results - Average Loss: 1.3725 | auc/_mean: 0.5169 | auc/healthy: 0.4783 | auc/multiple_diseases: 0.5885, auc/rust: 0.4543, auc/scab: 0.5467
Evaluating Results - Average Loss: 1.3556 | auc/_mean: 0.5417 | auc/healthy: 0.5330 | auc/multiple_diseases: 0.5749, auc/rust: 0.4295, auc/scab: 0.6296




Epoch: 04 | Time: 0m 35s
Training Results - Average Loss: 1.3656 | auc/_mean: 0.5186 | auc/healthy: 0.5290 | auc/multiple_diseases: 0.5506, auc/rust: 0.4558, auc/scab: 0.5390
Evaluating Results - Average Loss: 1.3527 | auc/_mean: 0.5444 | auc/healthy: 0.5340 | auc/multiple_diseases: 0.5812, auc/rust: 0.4453, auc/scab: 0.6170




Epoch: 05 | Time: 0m 35s
Training Results - Average Loss: 1.3588 | auc/_mean: 0.5192 | auc/healthy: 0.5101 | auc/multiple_diseases: 0.5201, auc/rust: 0.4831, auc/scab: 0.5634
Evaluating Results - Average Loss: 1.3475 | auc/_mean: 0.5546 | auc/healthy: 0.5443 | auc/multiple_diseases: 0.5909, auc/rust: 0.4604, auc/scab: 0.6229



...


Epoch: 196 | Time: 0m 35s
Training Results - Average Loss: 0.9596 | auc/_mean: 0.8304 | auc/healthy: 0.8864 | auc/multiple_diseases: 0.6315, auc/rust: 0.9064, auc/scab: 0.8973
Evaluating Results - Average Loss: 0.8235 | auc/_mean: 0.8948 | auc/healthy: 0.9479 | auc/multiple_diseases: 0.7019, auc/rust: 0.9734, auc/scab: 0.9561




Epoch: 197 | Time: 0m 35s
Training Results - Average Loss: 0.9516 | auc/_mean: 0.8239 | auc/healthy: 0.9011 | auc/multiple_diseases: 0.5912, auc/rust: 0.9072, auc/scab: 0.8961
Evaluating Results - Average Loss: 0.8227 | auc/_mean: 0.8947 | auc/healthy: 0.9483 | auc/multiple_diseases: 0.7008, auc/rust: 0.9733, auc/scab: 0.9563




Epoch: 198 | Time: 0m 35s
Training Results - Average Loss: 0.9523 | auc/_mean: 0.8342 | auc/healthy: 0.8866 | auc/multiple_diseases: 0.6417, auc/rust: 0.9103, auc/scab: 0.8984
Evaluating Results - Average Loss: 0.8212 | auc/_mean: 0.8945 | auc/healthy: 0.9477 | auc/multiple_diseases: 0.7012, auc/rust: 0.9731, auc/scab: 0.9561




Epoch: 199 | Time: 0m 35s
Training Results - Average Loss: 0.9600 | auc/_mean: 0.8261 | auc/healthy: 0.8880 | auc/multiple_diseases: 0.6163, auc/rust: 0.9065, auc/scab: 0.8937
Evaluating Results - Average Loss: 0.8203 | auc/_mean: 0.8942 | auc/healthy: 0.9473 | auc/multiple_diseases: 0.7004, auc/rust: 0.9728, auc/scab: 0.9562




Epoch: 200 | Time: 0m 35s
Training Results - Average Loss: 0.9616 | auc/_mean: 0.8191 | auc/healthy: 0.8817 | auc/multiple_diseases: 0.5963, auc/rust: 0.9058, auc/scab: 0.8926
Evaluating Results - Average Loss: 0.8171 | auc/_mean: 0.8933 | auc/healthy: 0.9474 | auc/multiple_diseases: 0.6961, auc/rust: 0.9733, auc/scab: 0.9563




In [10]:
model = get_model(config.model_name, config.num_classes)
model.load_state_dict(torch.load(f"{config.save_dirname}/best_model_epoch=200_loss=0.8171412737473197.pth"))
for param in model.parameters():
    param.requires_grad = True

criterion = LabelSmoothingCrossEntropy()
optimizer = AdamW(model.parameters(), lr=config.lr, weight_decay=1e-3)
num_train_steps = int(len(train_dataset) / config.batch_size * config.num_epochs)
scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=len(train_dataset)/config.batch_size*5, num_training_steps=num_train_steps)

Loaded pretrained weights for efficientnet-b4


In [11]:
trainer = Trainer(model, train_dataloader, valid_dataloader, criterion, optimizer, scheduler, device, config)
trainer.run();


Epoch: 01 | Time: 1m 43s
Training Results - Average Loss: 0.9827 | auc/_mean: 0.8086 | auc/healthy: 0.8684 | auc/multiple_diseases: 0.5975, auc/rust: 0.8900, auc/scab: 0.8784
Evaluating Results - Average Loss: 0.8259 | auc/_mean: 0.8933 | auc/healthy: 0.9480 | auc/multiple_diseases: 0.6952, auc/rust: 0.9744, auc/scab: 0.9557




Epoch: 02 | Time: 1m 41s
Training Results - Average Loss: 0.9405 | auc/_mean: 0.8376 | auc/healthy: 0.8895 | auc/multiple_diseases: 0.6467, auc/rust: 0.9161, auc/scab: 0.8982
Evaluating Results - Average Loss: 0.7597 | auc/_mean: 0.9053 | auc/healthy: 0.9643 | auc/multiple_diseases: 0.7078, auc/rust: 0.9859, auc/scab: 0.9632




Epoch: 03 | Time: 1m 41s
Training Results - Average Loss: 0.8836 | auc/_mean: 0.8403 | auc/healthy: 0.9098 | auc/multiple_diseases: 0.5903, auc/rust: 0.9441, auc/scab: 0.9167
Evaluating Results - Average Loss: 0.7020 | auc/_mean: 0.9153 | auc/healthy: 0.9733 | auc/multiple_diseases: 0.7254, auc/rust: 0.9917, auc/scab: 0.9707




Epoch: 04 | Time: 1m 41s
Training Results - Average Loss: 0.8451 | auc/_mean: 0.8632 | auc/healthy: 0.9210 | auc/multiple_diseases: 0.6675, auc/rust: 0.9452, auc/scab: 0.9191
Evaluating Results - Average Loss: 0.6587 | auc/_mean: 0.9210 | auc/healthy: 0.9786 | auc/multiple_diseases: 0.7366, auc/rust: 0.9942, auc/scab: 0.9745




Epoch: 05 | Time: 1m 41s
Training Results - Average Loss: 0.8032 | auc/_mean: 0.8738 | auc/healthy: 0.9335 | auc/multiple_diseases: 0.6703, auc/rust: 0.9593, auc/scab: 0.9322
Evaluating Results - Average Loss: 0.6263 | auc/_mean: 0.9251 | auc/healthy: 0.9818 | auc/multiple_diseases: 0.7453, auc/rust: 0.9953, auc/scab: 0.9778



...


Epoch: 124 | Time: 1m 41s
Training Results - Average Loss: 0.4380 | auc/_mean: 0.9966 | auc/healthy: 0.9984 | auc/multiple_diseases: 0.9936, auc/rust: 0.9965, auc/scab: 0.9978
Evaluating Results - Average Loss: 0.4518 | auc/_mean: 0.9843 | auc/healthy: 0.9979 | auc/multiple_diseases: 0.9417, auc/rust: 0.9999, auc/scab: 0.9977




Epoch: 125 | Time: 1m 41s
Training Results - Average Loss: 0.4324 | auc/_mean: 0.9977 | auc/healthy: 0.9987 | auc/multiple_diseases: 0.9975, auc/rust: 0.9978, auc/scab: 0.9970
Evaluating Results - Average Loss: 0.4520 | auc/_mean: 0.9832 | auc/healthy: 0.9979 | auc/multiple_diseases: 0.9377, auc/rust: 0.9999, auc/scab: 0.9975




Epoch: 126 | Time: 1m 41s
Training Results - Average Loss: 0.4281 | auc/_mean: 0.9969 | auc/healthy: 0.9986 | auc/multiple_diseases: 0.9911, auc/rust: 0.9986, auc/scab: 0.9991
Evaluating Results - Average Loss: 0.4536 | auc/_mean: 0.9851 | auc/healthy: 0.9981 | auc/multiple_diseases: 0.9448, auc/rust: 0.9999, auc/scab: 0.9976




Epoch: 127 | Time: 1m 40s
Training Results - Average Loss: 0.4305 | auc/_mean: 0.9968 | auc/healthy: 0.9990 | auc/multiple_diseases: 0.9910, auc/rust: 0.9983, auc/scab: 0.9990
Evaluating Results - Average Loss: 0.4532 | auc/_mean: 0.9869 | auc/healthy: 0.9981 | auc/multiple_diseases: 0.9520, auc/rust: 0.9999, auc/scab: 0.9976




Epoch: 128 | Time: 1m 41s
Training Results - Average Loss: 0.4312 | auc/_mean: 0.9966 | auc/healthy: 0.9990 | auc/multiple_diseases: 0.9931, auc/rust: 0.9964, auc/scab: 0.9978
Evaluating Results - Average Loss: 0.4512 | auc/_mean: 0.9874 | auc/healthy: 0.9981 | auc/multiple_diseases: 0.9539, auc/rust: 0.9999, auc/scab: 0.9976

EarlyStopping


In [10]:

class PlantDatasetTest(Dataset):
    
    def __init__(self, df, config, transforms=None):
    
        self.df = df
        self.images_dir = config.root_images
        self.class_names = config.class_names
        self.transforms=transforms
        
    def __len__(self):
        return self.df.shape[0]
    
    def __getitem__(self, idx):
        image_src = self.images_dir + self.df.iloc[idx]['image_id'] + '.jpg'
        image = cv2.imread(image_src, cv2.IMREAD_COLOR)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        
        if self.transforms:
            transformed = self.transforms(image=image)
            image = transformed['image']

        return image, idx

test_df = pd.read_csv(config.root + 'test.csv')
test_dataset = PlantDatasetTest(test_df, config, train_transforms)
test_dataloader = DataLoader(test_dataset, batch_size=config.batch_size, shuffle=False, num_workers=config.num_workers)

In [11]:
def predict(model, dataloader: DataLoader, device, dataset: Dataset, tta_count: int, config: ConfigExperiment):
    n_samples = len(dataset)
    y_probas_tta = np.zeros((n_samples, config.num_classes, tta_count), dtype=np.float32)
    
    model.eval()
    model.to(device)
    with torch.no_grad():
        for i_epoch in tqdm(range(tta_count), desc='TTA', total=tta_count, position=1, leave=True):
            for i, (images, idx) in enumerate(dataloader):
                images = images.to(device)
                outputs = model(images)
                y_pred = F.softmax(outputs, dim=1)
                
                tta_index = i_epoch - 1
                start_index = (i % len(dataloader)) * config.batch_size
                end_index = min(start_index + config.batch_size, n_samples)
                batch_y_probas = y_pred.detach().cpu().numpy()                
                y_probas_tta[start_index:end_index, :, tta_index] = batch_y_probas

    return y_probas_tta

In [12]:
model = get_model(config.model_name, config.num_classes)
model.load_state_dict(torch.load(f"{config.save_dirname}/best_model_epoch=128_auc_mean=0.9873604818428208.pth"))

Loaded pretrained weights for efficientnet-b4


<All keys matched successfully>

In [13]:
result = predict(model, test_dataloader, device, test_dataset, tta_count=5, config=config)

In [14]:
y_probas = np.mean(result, axis=-1)

In [15]:
y_probas[:10]

array([[0.07442209, 0.1672896 , 0.7393417 , 0.01894657],
       [0.02470513, 0.03390757, 0.8873944 , 0.05399287],
       [0.02042406, 0.02584796, 0.01119298, 0.94253504],
       [0.93904656, 0.02873715, 0.01980402, 0.01241229],
       [0.03448299, 0.10920537, 0.838239  , 0.01807265],
       [0.93136036, 0.03009171, 0.02716005, 0.01138789],
       [0.95543087, 0.01911015, 0.01621944, 0.00923952],
       [0.00922613, 0.19893962, 0.03346789, 0.7583663 ],
       [0.04018492, 0.7767425 , 0.02576838, 0.1573042 ],
       [0.02730695, 0.15143798, 0.7906407 , 0.03061433]], dtype=float32)

In [16]:
test_df = pd.read_csv(config.root + 'test.csv')
test_df["healthy"] = 0
test_df["multiple_diseases"] = 0
test_df["rust"] = 0
test_df["scab"] = 0
test_df[['healthy', 'multiple_diseases', 'rust', 'scab']] = y_probas
test_df.to_csv(config.submission_file, index=False)
test_df.head()

,image_id,healthy,multiple_diseases,rust,scab
0,Test_0,0.074422,0.167290,0.739342,0.018947
1,Test_1,0.024705,0.033908,0.887394,0.053993
2,Test_2,0.020424,0.025848,0.011193,0.942535
3,Test_3,0.939047,0.028737,0.019804,0.012412
4,Test_4,0.034483,0.109205,0.838239,0.018073


In [17]:
config.submission_file

'02_efficientnet-b4.csv'